## Deep Q-learning

클래스를 정의해서 CartPole을 구현한다.

* ReplayMemory

    transition을 저장하기 위한 메모리 클래스


* Agent

    CartPole의 수레에 해당
    update_Q_function과 get_action의 메서드를 가지고 있음
    Brain 클래스 객체를 멤버 변수로 가짐


* Brain

    Agent 클래스의 두뇌 역할
    딥러닝이 적용되는 부분이 Brain 부분
    Q 테이블을 이용해 Q-learning이 구현됨
    bins, digitize_state, update_Q_table, decide_action의 메서드를 가지고 있음
    bins, digitize_state: Agent가 관측한 생태 observation을 이산 변수로 변환


* Environment

    OpenAI Gym이 실행되는 실행 환경
    CartPole을 실행하며 실행을 맡을 run 메서드를 가지고 있음

Agent와 Brain을 별도의 클래스로 분리한 것은 딥러닝에서 Brain 클래스로 국한되기 때문이다.

## 구현 과정

* ReplayMemory 정의

    mini-batch 학습에서 학습된 데이터를 저장하는 역할을 할 ReplayMemory 클래스를 정의한다. ReplayMomory 클래스는 각 단계에서 해당 단계의 trainsition을 저장하는 push 메서드와 무작위로 선택된 transition을 꺼내오는 sample 메서드를 갖추고 있다. 그리고 len 메서드를 통해 현재 저장하고 있는 transition의 개수를 알려준다. 이 클래스는 저장된 transition의 개수가 capacity를 초과하면 오래된 것부터 지우고 transition의 인덱스를 새로운 transition에 부여한다.


* Action 결정 (Agent -> Brain -> Agent)

    * 행동을 결정하기 위해 Agent는 현재 상태 observation_t를 Brain 클래스에 전달한다.

    * Brain 클래스는 전달받은 상태변수를 이산변수로 변환한 다음 Q_table을 참조해서 행동을 결정한다

    * 결정된 현재 행동 action_t를 Agent에 전달한다.
    
    
* Action 실행 (Agent -> Environment -> Agent)

    * Agent는 Environment에 행동 action_t를 전달하며 Action을 취한 다음 Environment를 한 단계 진행시킨다.

    * Environment는 다시 행동 action_t를 실행한 결과가 되는 상태 observation_t+1과 이때 얻은 즉각보상 reward_t+1을 Agent에 반환한다.


* Q_table 수정 (Agent -> Brain)

    * Agent는 현재 상태 observation_t, 조금 전 취했던 행동 action_t, 행동의 결과로 얻은 새로운 상태 observation_t+1과 받게된 즉각보상 reward_t+1 네 개의 변수를 Brain에 전달한다.
    * Brain은 전달받은 변수를 통해 기존의 Q_table을 수정한다.
    * observation_t, action_t, observation_t+1, reward_t+1 네 개의 변수를 합쳐놓은 것을 'Transition'이라고 부른다.
    

Action 결정, Action 실행, Q_table 수정과 같이 세부과정을 포함한 총 세 개의 절차를 반복하며 Q_learning이 수행된다.

# 

## DQN 구현

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import gym

%matplotlib inline

In [2]:
'''
애니메이션 함수 정의
'''

from JSAnimation.IPython_display import display_animation
from matplotlib import animation
from IPython.display import display

def display_frames_as_gif(frames):
    
    plt.figure(figsize = (frames[0].shape[1] / 72.0, frames[0].shape[0] / 72.0), dpi = 72)
    patch = plt.imshow(frames[0])
    plt.axis('off')
    
    def animate(i):
        patch.set_data(frames[i])
        
    anim = animation.FuncAnimation(
        plt.gcf(), 
        animate, 
        frames = len(frames), 
        interval = 50
    )
    
    anim.save('CartPole_Q-learning_DQN.gif')
    display(display_animation(anim, default_mode = 'loop'))

In [3]:
'''
namedtuple 정의
'''

from collections import namedtuple

Transition = namedtuple('Transition', ('state', 'action', 'next_state', 'reward'))

In [4]:
'''
상수 정의
'''

# 태스크 네임
ENV = 'CartPole-v0'

# 각 상태를 이산변수로 변환할 구간 수
num_digitized = 6

# 시간할인율
gamma = 0.99 

# Epoch당 최대 단계 수
# CartPole-v0은 200단계를 버티면 클리어로 간주
max_steps = 200

# 최대 Epoch
nb_epochs = 1000

# batch size
batch_size = 32

# 메모리 최대 저장 수
capacity = 10000

In [5]:
'''
ReplayMemory 클래스 구현
'''

class ReplayMemory:
    
    def __init__(self, capacity):
        
        # 메모리 최대 저장 수
        self.capacity = capacity
        
        # 실제 transition을 저장할 변수
        self.memory = []
        
        # 저장 위치를 가리킬 인덱스 변수
        self.index = 0
        
    # transition = (state, action, state_next, reward)를 메모리에 저장
    def push(self, state, action, state_next, reward):
        
        if len(self.memory) < self.capacity:
            
            # 메모리가 여유로운 경우
            self.memory.append(None)
            
        # Transition이라는 namedtuple을 사용해 key-value 쌍의 형태로 출력을 저장
        self.memory[self.index] = Transition(state, action, state_next, reward)
        
        # 다음에 저장할 위치를 한 자리 뒤로 수정
        self.index = (self.index + 1) % self.capacity
    
    # batch_size 만큼 무작위로 저장된 transition을 추출
    def sample(self, batch_size):
        
        return random.sample(self.memory, batch_size)
    
    # __len__() 함수로 현재 저장된 transition의 개수를 반환
    def __len__(self):
        
        return len(self.memory)

In [6]:
'''
Agent 클래스 구현

생성자 메서드인 __init__()에서 상태변수의 수와 행동의 가짓수를 전달받고, 두뇌 역할을 할 Brain 클래스의 인스턴스를 만든다.
'''

# 실제 Agent의 역할을 하는 클래스
class Agent:
    
    def __init__(self, num_states, num_actions):
        
        # Agent의 행동을 결정하는 두뇌 역할 (Brain class 참조)
        self.brain = Brain(num_states, num_actions)
        
    def update_Q_function(self):
        
        # Brain에 전달 및 Q_table 수정 (Brain class 참조)
        self.brain.replay()
        
    def get_action(self, state, step):
        
        # 행동 결정 (Brain class 참조)
        action = self.brain.decide_action(state, step)
        
        return action
    
    # memory 인스턴스에 state, action, state_next, reward를 저장
    def memorize(self, state, action, state_next, reward):
        
        self.brain.memory.push(state, action, state_next, reward)

In [7]:
'''
Brain 클래스 구현

replay 메서드에서 Q_table을 신경망을 통해 수정하는 방법과 
device_action 메서드에서 다음 행동을 결정하는 방법은 epsilon-greedy 알고리즘을 이용하여 구현한다.

replay 메서드 역할
    1. 저장된 transition의 수를 확인
    2. mini-batch 생성
    3. 정답신호로 사용할 Q(s(t), a(t)) 계산
    4. 결합 가중치 수정
'''

import random
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F


# Agent의 두뇌 역할을 하는 클래스
class Brain:
    
    def __init__(self, num_states, num_actions):
        
        # 행동의 가짓수(왼쪽, 오른쪽)를 구함
        self.num_actions = num_actions
        
        # transition을 기억하기 위한 메모리 인스턴스 생성
        self.memory = ReplayMemory(capacity)
        
        self.model = nn.Sequential()
        self.model.add_module('fc1', nn.Linear(num_states, 32))
        self.model.add_module('relu1', nn.ReLU())
        self.model.add_module('fc2', nn.Linear(32, 32))
        self.model.add_module('relu2', nn.ReLU())
        self.model.add_module('fc3', nn.Linear(32, num_actions))
        
        # 신경망 구조출력
        print(self.model)
        
        # optimizer
        learning_rate = 0.0001
        self.optimizer = optim.Adam(self.model.parameters(), lr = learning_rate)
    
    # Experience Replay로 신경망 결합 가중치 학습
    def replay(self):
        
        # 저장된 transition의 수가 미니배치 크기보다 작으면 아무 것도 하지 않음
        if len(self.memory) < batch_size:
            return
        
        # 메모리 인스턴스에서 미니배치 추출
        transition = self.memory.sample(batch_size)
        
        # 각 변수를 미니배치에 맞는 형태로 변환
        # transition은 각 단계 별로 (state, action, state_next, reward) 형태로 batch_size 만큼 저장
        # (state, action, state_next, reward) * batch_size -> (state*batch_size, action*batch_size, state_next*batch_size, reward*batch_size)
        batch = Transition(*zip(*transition))
        
        # 각 변수의 요소를 미니배치에 맞게 변형한다.
        state_batch = torch.cat(batch.state)
        action_batch = torch.cat(batch.action)
        reward_batch = torch.cat(batch.reward)
        
        # 상태, 행동, 보상, non_final 상태로 된 미니배치를 나타내는 Variable을 생성
        non_final_next_states = torch.cat( [ s for s in batch.next_state if s is not None ] )
        
        # 신경망을 evaluation으로 변경
        self.model.eval()
        
        # self.model(state_batch)는 왼쪽, 오른쪽에 대한 Q값을 출력
        # [torch.FloatTensor of size batch_size * 2]의 형태
        # 실행한 행동 a(t)에 대한 Q값을 계산하기 때문에 
            # action_batch에서 취한 행동 a(t)가 왼쪽, 오른쪽인지에 대한 인덱스를 구하고 이에 대한 Q값을 gather 메서드로 모아온다
        state_action_values = self.model(state_batch).gather(1, action_batch)
        
        # max{Q(s_t+1, a)}값을 계산, 이때 다음 상태가 존재하는지 주의
        # CartPole이 done 상태가 아니고 next_state가 존재하는지 확인하는 인덱스 마스크 생성
        non_final_mask = torch.ByteTensor(tuple(map(lambda s: s is not None, batch.next_state)))
        
        # 전체 초기화
        next_state_values = torch.zeros(batch_size)
        
        # 다음 상태가 있는 인덱스에 대한 최개 Q값을 계산
        # 출력값에 접근하여 열 방향 최대값(max(1))이 되는 [값, 인덱스]를 구함
        # Q값(인덱스 0)을 출력
        # detach() 메서드로 Q값을 가져옴
        next_state_values[non_final_mask] = self.model(non_final_next_states).max(1)[0].detach()
        
        # Q(s(t), a(t))를 Q-learning으로 계산
        expected_state_action_values = reward_batch + gamma * next_state_values
        
        # 신경망을 train으로 변경
        self.model.train()
        
        # loss function은 Huber function 사용
        loss = F.smooth_l1_loss(state_action_values, expected_state_action_values.unsqueeze(1))
        
        # 경사 초기화
        self.optimizer.zero_grad()
        
        # 역전파
        loss.backward()
        
        # 가중치 수정
        self.optimizer.step()
        
    def decide_action(self, state, epoch):
        
        # epsilon-greedy 알고리즘 사용, 최적화된 행동의 비중을 증가시킴
        epsilon = 0.5 * (1 / ( epoch + 1) )
        
        if epsilon <= np.random.uniform(0, 1):
            
            # 신경망을 evaluation 모드로 변경
            self.model.eval()
            
            with torch.no_grad():
                
                # 신경망 출력의 최댓값에 대한 인덱스 = max(1)[1]
                # .view(1,1)은 [torch.LongTensor of size 1] 을 size 1*1로 변환하는 역할을 한다
                action = self.model(state).max(1)[1].view(1, 1)
                
        else:
            
            # 행동을 무작위로 반환 (0 또는 1)
            # action은 [torch.LongTensor of size 1*1] 형태가 된다
            action = torch.LongTensor([[random.randrange(self.num_actions)]])
            
        return action

In [8]:
'''
Environment 클래스 구현
'''

# OpenAI Gym이 실행되는 실행 환경 CartPole을 실행
class Environment:
    
    def __init__(self):
        
        # 실행할 태스크 설정
        self.env = gym.make(ENV)
        
        # 태스크의 상태 변수 수를 구함 (상태변수의 수: 4)
        num_states = self.env.observation_space.shape[0]
        
        # 가능한 행동 수를 구함
        num_actions = self.env.action_space.n
        
        # Agent 객체 생성
        self.agent = Agent(num_states, num_actions)
    
    # 실행
    def run(self):
        
        # 10epoch 동안 버틴 단계 수를 저장, 평균 step을 출력할 때 사용
        epoch_10_list = np.zeros(10)
        
        # 195단계 이상 버틴 epoch 수
        complete_epochs = 0
        
        # 마지막 epoch 여부
        is_epoch_final = False
        
        # 애니메이션을 만드는 데 사용할 이미지를 저장하는 변수
        frames = []
        
        # nb_epoch 만큼 반복
        for epoch in range(nb_epochs):
            
            # 환경 초기화
            observation = self.env.reset()
            
            # observation을 변환 없이 상태를 사용
            state = observation
            
            # Numpy 변수를 파이토치 텐서로 변환
            state = torch.from_numpy(state).type(torch.FloatTensor)
            
            # size 4를 size 1 * 4로 변환
            state = torch.unsqueeze(state, 0)
            
            # 각 epoch에 해당하는 반복
            for step in range(max_steps):
                
                # 마지막 에피소드면 frames에 각 단계의 이미지를 저장
                if is_epoch_final is True:
                    frames.append(self.env.render(mode = 'rgb_array'))
                    
                # 행동 선택
                action = self.agent.get_action(state, epoch)
                
                # 행동 action_t를 실행해 state_t+1, reward_t+1을 계산
                # reward, info는 사용하지 않으므로 under-bar로 처리
                observation_next, _, done, _ = self.env.step(action.item())
                
                # 보상 부여
                if done:
                    
                    # 다음 상태가 없으므로 None
                    state_next = None
                    
                    epoch_10_list = np.hstack( (epoch_10_list[1:], step + 1) )
                    
                    # 200단계를 넘어서거나 일정 각도 이상 기울면 done의 값이 True가 됨
                    if step < 195:
                        
                        # 봉이 쓰러지면 패널티 보상 -1 부여
                        reward = torch.FloatTensor([-1.0])
                        
                        # 195단계 이상 버티면 해당 epoch 성공 처리
                        complete_epochs = 0
                        
                    else:
                        
                        # 쓰러지지 않고 epoch를 끝내면 보상 1 부여
                        reward = torch.FloatTensor([1.0])
                        
                        # epoch 연속 성공 기록을 업데이트
                        complete_epochs = complete_epochs + 1
                        
                else:
                    
                    # epoch 중에는 보상 0 부여
                    reward = torch.FloatTensor([0.0])
                    
                    # observation 결과를 그대로 사용
                    state_next = observation_next
                    
                    # Numpy 변수를 파이토치 텐서로 변환
                    state_next = torch.from_numpy(state_next).type(torch.FloatTensor)
                    
                    # size 4를 size 1 * 4로 변환
                    state_next = torch.unsqueeze(state_next, 0)
                    
                    
                # 메모리에 경험을 저장
                self.agent.memorize(state, action, state_next, reward)
                    
                # Experience Raplay로 Q 함수를 업데이트
                self.agent.update_Q_function()
                
                # obervation 결과를 업데이트
                state = state_next
                
                # epoch 마무리
                if done:
                    print("{} Epoch was finished after {} time steps | The average step is {}".format(
                        epoch, step + 1, epoch_10_list.mean())
                         )
                    
                    break
                    
            # 마지막 epoch에서 애니메이션을 만들고 저장
            if is_epoch_final is True:
                display_frames_as_gif(frames)
                
                break
                
            # 10연속 이상 성공한 경우(195단계 이상 지속) 조기 종료
            if complete_epochs >= 10:
                print('10연속 이상 성공했습니다.')
                is_epoch_final = True

In [9]:
# 실행 엔트리 포인트
cartpole_env = Environment()
cartpole_env.run()

Sequential(
  (fc1): Linear(in_features=4, out_features=32, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=32, out_features=32, bias=True)
  (relu2): ReLU()
  (fc3): Linear(in_features=32, out_features=2, bias=True)
)
0 Epoch was finished after 19 time steps | The average step is 1.9
1 Epoch was finished after 10 time steps | The average step is 2.9
2 Epoch was finished after 10 time steps | The average step is 3.9
3 Epoch was finished after 10 time steps | The average step is 4.9
4 Epoch was finished after 9 time steps | The average step is 5.8
5 Epoch was finished after 9 time steps | The average step is 6.7
6 Epoch was finished after 10 time steps | The average step is 7.7
7 Epoch was finished after 8 time steps | The average step is 8.5
8 Epoch was finished after 9 time steps | The average step is 9.4
9 Epoch was finished after 10 time steps | The average step is 10.4
10 Epoch was finished after 10 time steps | The average step is 9.5
11 Epoch was finished after 8 time st

<ipython-input-7-e4851e6aa964>:89: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  ..\aten\src\ATen/native/IndexingUtils.h:20.)
  next_state_values[non_final_mask] = self.model(non_final_next_states).max(1)[0].detach()


15 Epoch was finished after 10 time steps | The average step is 9.3
16 Epoch was finished after 9 time steps | The average step is 9.2
17 Epoch was finished after 10 time steps | The average step is 9.4
18 Epoch was finished after 10 time steps | The average step is 9.5
19 Epoch was finished after 9 time steps | The average step is 9.4
20 Epoch was finished after 10 time steps | The average step is 9.4
21 Epoch was finished after 10 time steps | The average step is 9.6
22 Epoch was finished after 10 time steps | The average step is 9.7
23 Epoch was finished after 14 time steps | The average step is 10.2
24 Epoch was finished after 10 time steps | The average step is 10.2
25 Epoch was finished after 13 time steps | The average step is 10.5
26 Epoch was finished after 9 time steps | The average step is 10.5
27 Epoch was finished after 10 time steps | The average step is 10.5
28 Epoch was finished after 13 time steps | The average step is 10.8
29 Epoch was finished after 11 time steps | T

136 Epoch was finished after 40 time steps | The average step is 46.9
137 Epoch was finished after 73 time steps | The average step is 50.7
138 Epoch was finished after 120 time steps | The average step is 59.5
139 Epoch was finished after 34 time steps | The average step is 58.9
140 Epoch was finished after 76 time steps | The average step is 62.4
141 Epoch was finished after 75 time steps | The average step is 63.8
142 Epoch was finished after 62 time steps | The average step is 62.1
143 Epoch was finished after 36 time steps | The average step is 62.9
144 Epoch was finished after 42 time steps | The average step is 59.2
145 Epoch was finished after 44 time steps | The average step is 60.2
146 Epoch was finished after 68 time steps | The average step is 63.0
147 Epoch was finished after 40 time steps | The average step is 59.7
148 Epoch was finished after 33 time steps | The average step is 51.0
149 Epoch was finished after 78 time steps | The average step is 55.4
150 Epoch was finis

253 Epoch was finished after 134 time steps | The average step is 69.0
254 Epoch was finished after 104 time steps | The average step is 73.0
255 Epoch was finished after 200 time steps | The average step is 88.0
256 Epoch was finished after 200 time steps | The average step is 103.2
257 Epoch was finished after 137 time steps | The average step is 111.9
258 Epoch was finished after 91 time steps | The average step is 114.7
259 Epoch was finished after 55 time steps | The average step is 114.9
260 Epoch was finished after 49 time steps | The average step is 112.8
261 Epoch was finished after 49 time steps | The average step is 109.5
262 Epoch was finished after 48 time steps | The average step is 106.7
263 Epoch was finished after 34 time steps | The average step is 96.7
264 Epoch was finished after 26 time steps | The average step is 88.9
265 Epoch was finished after 24 time steps | The average step is 71.3
266 Epoch was finished after 24 time steps | The average step is 53.7
267 Epoc

MovieWriter ffmpeg unavailable; using Pillow instead.


368 Epoch was finished after 185 time steps | The average step is 198.5


c:\users\choih\appdata\local\programs\python\python38\lib\site-packages\JSAnimation\html_writer.py:281: MatplotlibDeprecationWarning: 
The 'clear_temp' parameter of setup() was deprecated in Matplotlib 3.3 and will be removed two minor releases later. If any parameter follows 'clear_temp', they should be passed as keyword, not positionally.
  super(HTMLWriter, self).setup(fig, outfile, dpi,


AttributeError: 'HTMLWriter' object has no attribute '_temp_names'